#### OBJETIVO NOTEBOOK
- Cálculo de los KPI's

# IMPORTAR LIBRERIAS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Importaciones de Scikit-learn
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

import xgboost as xgb

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# IMPORTAR DATASET

In [ ]:
df= pd.read_pickle('4_df_sp500_sin_nulos_sin_outliers_10022025.pkl')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5257 entries, 0 to 5297
Data columns (total 21 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ticker                                 5257 non-null   object 
 1   year                                   5257 non-null   int64  
 2   totalRevenue                           5257 non-null   float64
 3   depreciationAndAmortization            5257 non-null   float64
 4   ebitda                                 5257 non-null   float64
 5   netIncome                              5257 non-null   float64
 6   cashAndCashEquivalentsAtCarryingValue  5257 non-null   float64
 7   totalLiabilities                       5257 non-null   float64
 8   totalShareholderEquity                 5257 non-null   float64
 9   commonStockSharesOutstanding           5257 non-null   float64
 10  shortLongTermDebtTotal                 5257 non-null   float64
 11  capitalEx

In [ ]:
df.shape

(5257, 21)

AJUSTES FORMATO

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# Asegurar que la columna de año está ordenada
df = df.sort_values(by=['ticker', 'year'])

# TRANSFORMACIONES INICIALES

In [ ]:
# en 2016 para BKR tenemos un registro 'inf' a la variable ROIC que nos da problemas. EL motivo es que durante el momento de la constitución, una empresa no suele tener liabilities (PASIVO), y entonces no podemos calcular el ROIC porque no se puede dividir 0/X.
# eliminaremos este año de esta empresa ya que no tenemos datos suficientes para sacar conclusiones
df[df['ticker']=='BKR']

,ticker,year,totalRevenue,depreciationAndAmortization,ebitda,netIncome,cashAndCashEquivalentsAtCarryingValue,totalLiabilities,totalShareholderEquity,commonStockSharesOutstanding,...,capitalExpenditures,cashflowFromInvestment,dividendPayout,cashflowFromFinancing,operatingCashflow,Year-End Price,sector,totalAssets,totalCurrentAssets,totalCurrentLiabilities
645,BKR,2016,25330000000.00,257000000.00,2574000000.00,1943000000.00,981000000.00,0.00,14688000000.00,430000000.00,...,989000000.00,-817000000.00,0.00,-1592000000.00,3062000000.00,58.20,Energy,21721000000.00,7401000000.00,4934000000.00
638,BKR,2017,17128000000.00,387000000.00,103000000.00,-103000000.00,7030000000.00,42223000000.00,14277000000.00,1129000000.00,...,665000000.00,-4123000000.00,155000000.00,10919000000.00,-799000000.00,25.85,Energy,56500000000.00,18424000000.00,9227000000.00
639,BKR,2018,22825000000.00,455000000.00,1156000000.00,195000000.00,2515000000.00,34974000000.00,17465000000.00,1035000000.00,...,995000000.00,-578000000.00,315000000.00,-4363000000.00,1762000000.00,17.98,Energy,52439000000.00,14971000000.00,9020000000.00
640,BKR,2019,23463000000.00,365000000.00,1439000000.00,128000000.00,2147000000.00,31440000000.00,21929000000.00,1027000000.00,...,1240000000.00,-1045000000.00,395000000.00,-1534000000.00,2126000000.00,22.11,Energy,53369000000.00,15222000000.00,10014000000.00
641,BKR,2020,20088000000.00,308000000.00,-15670000000.00,-9940000000.00,3445000000.00,25114000000.00,12893000000.00,1035000000.00,...,974000000.00,-618000000.00,488000000.00,225000000.00,1304000000.00,18.77,Energy,38007000000.00,16455000000.00,10227000000.00
642,BKR,2021,20203000000.00,253000000.00,1563000000.00,-219000000.00,3252000000.00,20478000000.00,14830000000.00,2059000000.00,...,856000000.00,-463000000.00,592000000.00,-2143000000.00,2374000000.00,22.33,Energy,35308000000.00,15065000000.00,9128000000.00
643,BKR,2022,20453000000.00,222000000.00,1407000000.00,-601000000.00,1883000000.00,19787000000.00,14394000000.00,2017000000.00,...,989000000.00,-1564000000.00,726000000.00,-1592000000.00,1888000000.00,28.11,Energy,34181000000.00,14592000000.00,11075000000.00
644,BKR,2023,25330000000.00,257000000.00,2574000000.00,1943000000.00,2646000000.00,21577000000.00,15368000000.00,2000000000.00,...,1224000000.00,-817000000.00,786000000.00,-2028000000.00,3062000000.00,33.33,Energy,36945000000.00,16301000000.00,12991000000.00


In [ ]:
df = df[~((df['year'] == 2016) & (df['ticker'] == 'BKR'))]

# CREACIÓN KPIs

In [ ]:
    # ROE (Return on Equity)
    df['ROE'] = df['netIncome'] / df['totalShareholderEquity']

    # ROIC
    df['ROIC'] = (df['netIncome'] - df['dividendPayout']) / (df['totalLiabilities'] / df['totalShareholderEquity'])

    # Debt-to-Equity ratio
    df['Debt_to_Equity'] = df['totalLiabilities'] / df['totalShareholderEquity']

    # P/E Ratio (Price-to-Earnings)
    df['Earnings_Per_Share'] = df['netIncome'] / df['commonStockSharesOutstanding']
    df['P/E_Ratio'] = df['Year-End Price'] / df['Earnings_Per_Share']

    # Net Income Margin
    df['Net_Income_Margin'] = df['netIncome'] / df['totalRevenue']

    # P/S Ratio (Price-to-Sales)
    df['Revenue_Per_Share'] = df['totalRevenue'] / df['commonStockSharesOutstanding']
    df['P/S_Ratio'] = df['Year-End Price'] / df['Revenue_Per_Share']

    # Market Cap
    df['Market_Cap'] = df['commonStockSharesOutstanding'] * df['Year-End Price']

    # Returns (Yearly Return)
    df['Yearly_Return'] = df.groupby('ticker')['Year-End Price'].pct_change()

    # Calcular el return de los últimos 5 años (5Y) usando transform para evitar problemas de índice
    df['return_5y'] = df.groupby('ticker')['Year-End Price'].transform(lambda x: x.pct_change(periods=5))

    # Crecimiento en ventas
    df['Sales_Growth'] = df.groupby('ticker')['totalRevenue'].pct_change()

    # Crecimiento en precio
    df['Price_Growth'] = df.groupby('ticker')['Year-End Price'].pct_change()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5256 entries, 0 to 5297
Data columns (total 34 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ticker                                 5256 non-null   object 
 1   year                                   5256 non-null   int64  
 2   totalRevenue                           5256 non-null   float64
 3   depreciationAndAmortization            5256 non-null   float64
 4   ebitda                                 5256 non-null   float64
 5   netIncome                              5256 non-null   float64
 6   cashAndCashEquivalentsAtCarryingValue  5256 non-null   float64
 7   totalLiabilities                       5256 non-null   float64
 8   totalShareholderEquity                 5256 non-null   float64
 9   commonStockSharesOutstanding           5256 non-null   float64
 10  shortLongTermDebtTotal                 5256 non-null   float64
 11  capitalEx

In [ ]:
# Especifica la ruta en tu Google Drive
ruta_guardado = '/content/drive/MyDrive/ACADEMIC/MASTER DATA SCIENCE & AI (NUCLIO DIGITAL SCHOOL)/TFM/data01032025.csv'

In [ ]:
# Guardar con separador decimal como coma
df.to_csv(ruta_guardado, index=False, sep=';', decimal=',')

In [ ]:
ruta_guardado_pkl ='/content/drive/MyDrive/ACADEMIC/MASTER DATA SCIENCE & AI (NUCLIO DIGITAL SCHOOL)/TFM/data01032025.pkl'

In [ ]:
df.to_pickle(ruta_guardado_pkl)

In [ ]:
df.head()

,ticker,year,totalRevenue,depreciationAndAmortization,ebitda,netIncome,cashAndCashEquivalentsAtCarryingValue,totalLiabilities,totalShareholderEquity,commonStockSharesOutstanding,...,Earnings_Per_Share,P/E_Ratio,Net_Income_Margin,Revenue_Per_Share,P/S_Ratio,Market_Cap,Yearly_Return,return_5y,Sales_Growth,Price_Growth
0,A,2013,3894000000.00,199000000.00,1098000000.00,724000000.00,2675000000.00,5397000000.00,5286000000.00,602000000.00,...,1.20,31.38,0.19,6.47,5.83,22715972835.34,NaN,NaN,NaN,NaN
1,A,2014,4048000000.00,197000000.00,808000000.00,504000000.00,2218000000.00,5511000000.00,5301000000.00,608000000.00,...,0.83,45.47,0.12,6.66,5.66,22917108510.40,-0.00,NaN,0.04,-0.00
2,A,2015,4038000000.00,156000000.00,665000000.00,401000000.00,2003000000.00,3309000000.00,4167000000.00,611000000.00,...,0.66,59.42,0.10,6.61,5.90,23828127554.30,0.03,NaN,-0.00,0.03
3,A,2016,4202000000.00,152000000.00,768000000.00,462000000.00,2289000000.00,3548000000.00,4243000000.00,614000000.00,...,0.75,57.09,0.11,6.84,6.28,26375204751.58,0.10,NaN,0.04,0.10
4,A,2017,4472000000.00,120000000.00,1002000000.00,684000000.00,2678000000.00,3591000000.00,4831000000.00,322000000.00,...,2.12,29.99,0.15,13.89,4.59,20515623726.06,0.48,NaN,0.06,0.48
